# Energa Solar Production Report Analysis

The script will analyse energy consumption and energy production reports generated by Polish operator Energa SA.

## Getting the reports

Login into Energa SA web portal and download the energy consumption and energy production CSV reports. The CSV file looks as follows:

```csv
Data i godzina;ENERGIA POBRANA [KWH];ENERGIA WPROWADZONA [KWH];SALDO ENERGII (BILANS = ENERGIA POBRANA - ENERGIA WPROWADZONA) [KWH] 
01.01.2024 01:00;0,378;0;0,378
01.01.2024 02:00;0,199;0;0,199
...
```

Place all downloaded CSV files into the `data` directory.

## Install Python libraries

The notebook requires the following libraries to be installed:

```sh
pip install pandas
pip install matplotlib
pip install seaborn
```

Import libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import glob

Read the data from `data/*.csv`

In [2]:
all_files = glob.glob("data/*.csv")
df = pd.concat((pd.read_csv(f, sep=";", decimal=",", 
                 parse_dates=["Data i godzina"], 
                 encoding='utf-8') for f in all_files), ignore_index=True)


Preview dataframe.

In [ ]:
df.head()

Parse 'Data i godzina' into datetime, set it as index, rename long column names to shorter ones.

In [4]:
# Parse date time correctly
df['Data i godzina'] = pd.to_datetime(df['Data i godzina'], format='%d.%m.%Y %H:%M')
# Set "Data i godzina" as the index
df.set_index("Data i godzina", inplace=True)
# Rename columns
df = df.rename(columns={'ENERGIA POBRANA [KWH]': 'ENERGIA POBRANA', 'ENERGIA WPROWADZONA [KWH]': 'ENERGIA WPROWADZONA', 'SALDO ENERGII (BILANS = ENERGIA POBRANA - ENERGIA WPROWADZONA) [KWH] ': 'SALDO ENERGII'})

Preview the update dataframe.

In [ ]:
df.head()

Group data by year and calculate sums for energy produced, consumed, and their difference.

In [ ]:
# Group data by year and calculate sums for energy produced, consumed, and their difference
df_grouped = df.groupby(pd.Grouper(freq="YE")).agg(
    {'ENERGIA POBRANA': 'sum', 'ENERGIA WPROWADZONA': 'sum'}
)
df_grouped['SALDO ENERGII'] = df_grouped['ENERGIA WPROWADZONA'] - df_grouped['ENERGIA POBRANA']

# Print the summary
print(df_grouped)

Now break down the data by month.

In [ ]:
# Group data by month and calculate sums for energy produced, consumed, and their difference
df_grouped = df.groupby(pd.Grouper(freq="MS")).agg(
    {'ENERGIA POBRANA': 'sum', 'ENERGIA WPROWADZONA': 'sum'}
)
df_grouped['SALDO ENERGII'] = df_grouped['ENERGIA WPROWADZONA'] - df_grouped['ENERGIA POBRANA']

# Print the summary
print(df_grouped)

Get top 10 days with biggest consumption and biggest production.

In [ ]:
df_grouped = df.groupby(pd.Grouper(freq="B")).agg(
    {'ENERGIA POBRANA': 'sum', 'ENERGIA WPROWADZONA': 'sum'}
)
df_grouped['SALDO ENERGII'] = df_grouped['ENERGIA WPROWADZONA'] - df_grouped['ENERGIA POBRANA']

# Get 10 days with biggest consumptions
top_10_consumption = df_grouped.nlargest(10, 'ENERGIA POBRANA')

# Get 10 days with biggest production
top_10_production = df_grouped.nlargest(10, 'ENERGIA WPROWADZONA')

# Get 10 days with lowest consumptions
bottom_10_consumption = df_grouped.nsmallest(10, 'ENERGIA POBRANA')

# Get 10 days with lowest production
bottom_10_production = df_grouped.nsmallest(10, 'ENERGIA WPROWADZONA')


print("10 days with biggest consumptions:")
print(top_10_consumption)

print("\n10 days with lowest consumption:")
print(bottom_10_consumption)

print("\n10 days with biggest production:")
print(top_10_production)

print("10 days with lowest production:")
print(bottom_10_production)


Plot the energy consumption and energy production by week.

In [ ]:
df_grouped = df.groupby(pd.Grouper(freq="W")).agg(
    {"ENERGIA POBRANA": "sum", "ENERGIA WPROWADZONA": "sum"}
)

# Create line plot with consumption and production
sns.lineplot(data=df_grouped, x=df_grouped.index, y="ENERGIA POBRANA", label="Energy consumed")
sns.lineplot(data=df_grouped, x=df_grouped.index, y="ENERGIA WPROWADZONA", label="Energy produced")

# Set plot title, labels, and legend
plt.title("Energy consumed vs. energy produced (weekly)")
plt.xlabel("Date")
plt.ylabel("Energy [kWh]")
plt.legend()

# Show the plot
plt.show()

Plot the distribution of energy consumed and energy produced by hour.

In [ ]:
# Extract hour from the index
df['Hour'] = df.index.hour

# Group data by hour and calculate sums for energy produced and consumed
df_grouped = df.groupby('Hour').agg(
    {'ENERGIA POBRANA': 'sum', 'ENERGIA WPROWADZONA': 'sum'}
)

# Create a bar plot with consumption as negative numbers
sns.barplot(x=df_grouped.index, y=df_grouped['ENERGIA WPROWADZONA'], label='Energy produced', color='orange')
sns.barplot(x=df_grouped.index, y=-df_grouped['ENERGIA POBRANA'], label='Energy consumed')

# Set plot title, labels, and legend
plt.title('Distribution of energy consumed and produced by hour')
plt.xlabel('Hour')
plt.ylabel('Energy [kWh]')
plt.legend()

# Show the plot
plt.show()

Now plot the distribution by month. You can observe the difference between winter and summer months.

In [ ]:
# Extract hour and month from the index
df['Hour'] = df.index.hour
df['Month'] = df.index.month

# Group data by hour and month, calculate sums for energy produced and consumed
df_grouped = df.groupby(['Hour', 'Month']).agg(
    {'ENERGIA POBRANA': 'sum', 'ENERGIA WPROWADZONA': 'sum'}
)

# Iterate through each month and plot the data
for month in df_grouped.index.get_level_values('Month').unique():
    month_data = df_grouped.xs(month, level='Month')

    # Create a bar plot
    sns.barplot(x=month_data.index, y=month_data['ENERGIA WPROWADZONA'], label='Energy produced', color='orange')
    sns.barplot(x=month_data.index, y=-month_data['ENERGIA POBRANA'], label='Energy consumed')

    # Set plot title, labels, and legend
    plt.title(f'Distribution of energy consumed and produced by hour - {calendar.month_name[month]}')
    plt.xlabel('Hour')
    plt.ylabel('Energy [kWh]')
    plt.legend()

    # Save the plot as an image
    # plt.savefig(f'energy_consumption_production_month_{month}.png')
    plt.show()
    plt.clf()  # Clear the plot for the next iteration

Distribution of hours with zero energy consumption. Usually these are peak sunlight hours.

In [ ]:
zero_consumption_hours = df[df['ENERGIA POBRANA'] == 0].index.hour

# Count occurrences of each hour
hour_counts = zero_consumption_hours.value_counts()

# Create a bar plot
plt.bar(hour_counts.index, hour_counts)
plt.xlabel('Hour')
plt.ylabel('Number of Occurrences')
plt.title('Hours with Zero Energy Consumption')
plt.show()

Distribution of hours with zero energy production. Usually these are late evening and night hours.

In [ ]:
# Filter for hours when 'ENERGIA WPROWADZONA' is 0
zero_production_hours = df[df['ENERGIA WPROWADZONA'] == 0].index.hour

# Count occurrences of each hour
hour_counts = zero_production_hours.value_counts()

# Create a bar plot
plt.bar(hour_counts.index, hour_counts)
plt.xlabel('Hour')
plt.ylabel('Number of Occurrences')
plt.title('Hours with Zero Energy Production')
plt.show()